# 7. 코사인 유사도 & Clustering 

In [1]:
from urllib.parse import quote_plus

import requests
import lxml.html

In [2]:
search_url = ('http://section.blog.naver.com/sub/SearchBlog.nhn?type=post&option.keyword={query}'
              '&option.page.currentPage={page}&option.orderBy=sim')

In [3]:
query = quote_plus('딥러닝')
query

'%EB%94%A5%EB%9F%AC%EB%8B%9D'

In [4]:
url = search_url.format(query=query, page=2)
url

'http://section.blog.naver.com/sub/SearchBlog.nhn?type=post&option.keyword=%EB%94%A5%EB%9F%AC%EB%8B%9D&option.page.currentPage=2&option.orderBy=sim'

In [5]:
res = requests.get(url)

In [6]:
root = lxml.html.fromstring(res.text)

In [7]:
root.cssselect('h5 a')

[<Element a at 0x1bdc61a0c28>,
 <Element a at 0x1bdc61a0c78>,
 <Element a at 0x1bdc61a0cc8>,
 <Element a at 0x1bdc61a0d18>,
 <Element a at 0x1bdc61a0d68>,
 <Element a at 0x1bdc61a0db8>,
 <Element a at 0x1bdc61a0e08>,
 <Element a at 0x1bdc61a0e58>,
 <Element a at 0x1bdc61a0ea8>,
 <Element a at 0x1bdc61a0ef8>]

 - a Class가 존재하지 않는다. 그럴때는 위에 태그를 본다. 

In [8]:
for link in root.cssselect('h5 a'):
    print(link.text_content(), link.attrib['href'])

인공지능, 머신러닝, 딥러닝 입문 http://patent21.blog.me/220976563267
딥러닝을 위한 라이브러리 TensorFlow http://blog.naver.com/rkdwnsdud555?Redirect=Log&logNo=220988902977&from=section
[도서]밑바닥부터 시작하는 딥러닝 http://blog.naver.com/civilize?Redirect=Log&logNo=220976431562&from=section
[딥러닝] 네이버의 쿼리 어시스턴트 http://blog.naver.com/chatton03?Redirect=Log&logNo=220969790092&from=section
머신러닝 딥러닝 학습을 위한 GPGPU 워크스테이션 http://blog.naver.com/wellmpc?Redirect=Log&logNo=220971162452&from=section
밑바닥부터 시작하는 딥러닝 http://blog.naver.com/hyongsu44?Redirect=Log&logNo=220988314262&from=section
딥러닝(Deep Learning) 실습과정 강좌 오픈! http://blog.officen.kr/220953268153
우분투 딥러닝 학습을 위한 4Way GPU 워크스테이션 http://blog.naver.com/wellmpc?Redirect=Log&logNo=220962680123&from=section
74.인공지능과 딥러닝 [2017.01.**~****.02.03] http://blog.naver.com/dudwo7970?Redirect=Log&logNo=220983998483&from=section
[ML] 테리의 딥러닝 토크 (동영상) http://horajjan.blog.me/220961185493


## URL 분해
 - 크롤링에서 필요했던 내용들을 분해해서 데이터를 format으로 하면 된다. 

In [9]:
from urllib.parse import parse_qs, urlparse

In [10]:
result = urlparse('http://blog.naver.com/civilize?Redirect=Log&logNo=220976431562&from=section')

In [11]:
result

ParseResult(scheme='http', netloc='blog.naver.com', path='/civilize', params='', query='Redirect=Log&logNo=220976431562&from=section', fragment='')

In [12]:
result.netloc

'blog.naver.com'

In [13]:
result.path

'/civilize'

In [14]:
result.query

'Redirect=Log&logNo=220976431562&from=section'

In [15]:
qs = parse_qs(result.query)
qs

{'Redirect': ['Log'], 'from': ['section'], 'logNo': ['220976431562']}

In [16]:
qs['logNo'][0]

'220976431562'

In [17]:
post_url = 'http://blog.naver.com/PostView.nhn?blogId={}&logNo={}'.format(result.path[1:], qs['logNo'][0])

In [18]:
post_url

'http://blog.naver.com/PostView.nhn?blogId=civilize&logNo=220976431562'

## 게시물 내용 가져오기 

In [19]:
post_res = requests.get(post_url)
post_root = lxml.html.fromstring(post_res.text)

In [20]:
post_root.cssselect('div#postViewArea')[0].text_content()

'\r\n\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\r\n \r\n \r\n  밑바닥부터 시작하는 딥러닝 \r\n  \r\n  작가\r\n  사이토 고키\r\n  출판\r\n  한빛미디어\r\n  발매\r\n  2017.01.03.\r\n  평점\r\n  \r\n   \r\n    \r\n   \r\n  \r\n  \r\n  리뷰보기\r\n \r\n 이런 책이 진작 나왔다면 내가 머신러닝 공부하는데 장벽을 좀 덜 느꼈을텐데 말이다.왜 2017년에서야 나온거냐.일단 책 내용은 요즘 굉장히 핫한 뉴럴넷, 딥러닝을 파이썬으로 구현해 보면서 익히는 것이고,책 제목대로 초짜들도 따라하면서 뉴럴넷의 원리와 필요한 지식들을 배울 수 있게 구성되어 있다.뭐 그래도 calculus, linear algebra 정도는 머릿속에 배경지식을 깔고 들어가야 하고,애초에 머신러닝 분야가 통계나 수학적인 백그라운드 없으면 쉽게 할 수 없으니까그정도는 공부하고 읽는것을 추천한다.근데 난 이 내용 석사때 분명 배우긴 배웠고, matlab으로 구현까지 했던 기억이 있는데하도 손을 놔서 그런지 다 까묵어서 리마인드하는 용도로 보고 있기는 하다.그런 용도로도 괜찮은 책인것도 같고.\r\n\t\t\t\t\t\t\r\n\t\t\t\t\t\t'

## 블로그 스크래핑 

In [21]:
import tqdm
from urllib.parse import urljoin

In [22]:
keyword = '딥러닝'
query = quote_plus(keyword)

In [23]:
search_url = ('http://section.blog.naver.com/sub/SearchBlog.nhn?type=post&option.keyword={query}'
              '&option.page.currentPage={page}&option.orderBy=sim')

In [24]:
posts = []
for page in tqdm.tqdm_notebook(range(1, 20)):
    url = search_url.format(query=query, page=page)
    res = requests.get(url)
    root = lxml.html.fromstring(res.text)
    
    for link in root.cssselect('h5 a'):
        link_url = link.attrib['href']

        # 다른 형식의 주소는 무시
        if not link_url.startswith('http://blog.naver.com'):
            continue

        # 진짜 주소
        result = urlparse(link_url)
        blog_id = result.path[1:]
        qs = parse_qs(result.query)
        post_id = qs['logNo'][0]
        post_url = 'http://blog.naver.com/PostView.nhn?blogId={}&logNo={}'.format(blog_id, post_id)
        
        # 본문 가져오기
        post_res = requests.get(post_url)
        post_root = lxml.html.fromstring(post_res.text)
        
        try:
            body = post_root.cssselect('div#postViewArea')[0]
            posts.append(body.text_content())
        except IndexError:
            continue

In [25]:
len(posts)

88

## CSV로 저장 

In [26]:
import csv
import re 

In [27]:
with open('posts.csv', 'w', encoding='utf8', newline='') as f:
    w = csv.writer(f)
    for post in posts:
        post_short = re.sub(r'\s+', ' ', post)  # 모든 종류의 공백을 빈 칸 하나로 바꿈 (엑셀에서 보기 좋게)
        w.writerow([post_short])

## CSV에서 불러오기 

In [1]:
import csv

In [3]:
posts = []
with open("posts.csv", encoding="cp949") as f:
    reader = csv.reader(f)
    for row in reader:
        post = row[0]
        if len(post) > 100:
            posts.append(post)

In [4]:
len(posts)

534

In [5]:
posts[:2]

[' 머신러닝의 딥러닝, 미래사회를 이끌다 산업 현장의 지각 변동을 이룰 미래 기술 미국의 IT 리서치 기업 가트너 그룹은 지난 2016년, ITxpo 2016에서 향후 5년간 혁신 잠재력을 지닌 10대 전략기술 트렌드를 발표했다. 가트너가 선정한 2017년 10대 전략 기술은 인텔리전트(Intellegent)와 디지털(Digital), 그리고 메시(Mesh)라는 3가지 영역으로 구분된다. 그중 인공지능과 고급 기계 학습, 즉 머신러닝은 10대 기술의 서두로서 로봇, 자율주행자동차, 가전 기기 등 물리적 디바이스를 비롯해 미래 IT 산업을 선도할 핵심기술로 평가 받고 있다.세상을 바꿀 최고의 전략 기술지난 2016년 ‘ITxpo 2016’에서 미국의 가트너 그룹이 발표한 ‘2017년 10대 전략기술 트렌드’는 향후 5년간 IT 산업 발전을 이끌 10가지 핵심 기술에 대해 이야기한다. 그중 인텔리전트(Intellegent), 즉 지능의 영역으로 분류되는 ‘인공지능과 고급 기계 학습(AI & Advanced Machine Learning)’은 기존의 알고리즘을 넘어 학습 예측, 적응을 비롯해 잠재적으로 스스로 가동하는 자율시스템을 만들어 지능형 스마트 기기를 만든다.인공지능의 한 분야인 머신러닝(Machine Learning)은 데이터의 생성량과 주기, 형식 등 방대한 빅데이터들을 분석해 미래를 예측하는 기술을 말한다. 이 기술은 기존의 빅데이터 분석과 유사점을 지녔지만, 데이터를 수집·분석해 미래를 예측한다는 점에서 차이점을 지녔다. 딥러닝, 신경망, 자연어처리, 첨단기법 등 기술의 결합으로 완성된 머신러닝 기술은 최근 각광받고 있는 물리적 디바이스(로봇, 자율주행자동차 등)을 비롯해 지능형 앱, 메시 디바이스 등 다양한 분야에 적용·융합될 혁신 기술로 평가받는다. 대표적 사례로 글로벌 소셜 네트워크 서비스를 제공하는 페이스북은 ‘Project PANDA’를 통한 딥페이스 기술을 연구한다. 또한, 세계 최대 비디오 스트리밍 기업 넷플릭스는 사용자의 구매 이력을 

## TDM 만들기 

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Komoran

In [7]:
tagger = Komoran()

In [8]:
def get_noun(text):
    nouns = tagger.nouns(text)
    return [n for n in nouns if len(n) > 1]  # 2글자 이상인 명사만 추출

In [9]:
cv = CountVectorizer(tokenizer=get_noun, max_features=500)

In [10]:
tdm = cv.fit_transform(posts)

## TDM 저장 

In [11]:
import numpy as np 

In [12]:
np.savez('blog_tdm.npz',tdm)

## 단어 목록 저장 

In [13]:
import json

In [14]:
with open('blog_tdm.json', "w", encoding="utf8") as f:
    json.dump(cv.get_feature_names(),f)

## TDM 불러오기 

In [15]:
data = np.load('blog_tdm.npz', encoding='latin1')

In [16]:
data.files

['arr_0']

In [17]:
tdm = data['arr_0'].item()

In [18]:
tdm

<534x500 sparse matrix of type '<class 'numpy.int64'>'
	with 26169 stored elements in Compressed Sparse Row format>

## 단어 목록 불러오기 

In [20]:
with open('blog_tdm.json', encoding='utf8') as f:
    words = json.load(f)

In [21]:
words[:10]

['가격', '가중치', '가지', '가치', '감사', '감소', '감정', '강의', '강좌', '강화']

## 차원 축소 

In [22]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

In [23]:
svd = TruncatedSVD(n_components=10)

In [24]:
pos = svd.fit_transform(tdm)

In [25]:
pos.shape

(534, 10)

In [27]:
tdm # 534파일을 500개 단어로 표현 했었다. 그걸 위에서 10개 차원으로 변경 

<534x500 sparse matrix of type '<class 'numpy.int64'>'
	with 26169 stored elements in Compressed Sparse Row format>

In [28]:
pos[0, :]

array([ 34.0023001 , -10.80033207,   3.64908666, -23.39763816,
        -6.00303814,  14.37208342,   1.65445348, -13.37121719,
         3.9279552 ,   1.18865712])

## 정규화 
 - 길이도 정보를 가지고 있는데 왜 줄이느냐 라는 비판도 있다.
  - 보통 많이 사용한다. 
  - 다른걸 한다고 해서 크게 좋아지는건 없다...

In [38]:
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer) 
# 연속으로 적용할 수 있도록 svd -> normalizer가 될 수 있도록 

In [30]:
pos = lsa.fit_transform(tdm)

In [31]:
pos.shape

(534, 10)

In [32]:
pos[0, :]

array([ 0.7127271 , -0.22638801,  0.07648591, -0.49043818, -0.12589737,
        0.30123156,  0.03486466, -0.28154767,  0.0818862 ,  0.0246629 ])

In [33]:
sum(pos[0, :] ** 2)

1.0000000000000002

## 클러스터링 
 - 1차로 임의의 점으로 cluster 갯수로 해서 가까운 놈들로 나눈다.
 - cluster 에 나누어진 놈들끼리 평균을 구하면 다시 중심을 구할 수 있다.
 - 또 다시 평균을 내서 중심을 잡는다. 
 - 그리고 점들을 중심에서의 거리를 구하고 가까운 놈들에게 포함시키게 한다.
 - 변화가 없을때 까지 돌린다. 

In [34]:
from sklearn.cluster import KMeans

In [35]:
km = KMeans(n_clusters=5)

In [36]:
km.fit(pos)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [37]:
km.labels_

array([0, 1, 4, 1, 3, 3, 2, 4, 1, 1, 1, 3, 0, 0, 4, 4, 4, 1, 4, 3, 3, 4, 2,
       2, 2, 3, 2, 4, 4, 1, 3, 2, 0, 3, 3, 0, 2, 3, 0, 0, 1, 0, 1, 2, 3, 1,
       3, 4, 1, 1, 1, 4, 3, 3, 3, 1, 3, 0, 2, 4, 2, 2, 3, 3, 2, 0, 2, 3, 2,
       2, 0, 1, 1, 1, 0, 1, 2, 1, 3, 3, 3, 1, 2, 3, 3, 3, 0, 3, 0, 4, 2, 3,
       0, 3, 0, 2, 3, 2, 3, 0, 3, 2, 1, 0, 4, 2, 0, 1, 3, 0, 2, 1, 3, 3, 4,
       3, 3, 1, 3, 4, 0, 2, 1, 0, 0, 0, 1, 2, 3, 3, 0, 3, 1, 4, 1, 0, 0, 3,
       1, 0, 4, 2, 2, 2, 1, 2, 0, 3, 3, 1, 1, 2, 1, 1, 4, 3, 0, 1, 1, 3, 1,
       1, 0, 0, 4, 0, 0, 1, 1, 1, 0, 2, 1, 3, 1, 1, 3, 0, 3, 3, 3, 3, 3, 2,
       4, 4, 2, 2, 3, 1, 0, 0, 3, 1, 0, 3, 0, 3, 4, 1, 2, 1, 3, 4, 4, 4, 3,
       1, 0, 0, 3, 3, 0, 2, 3, 3, 0, 1, 1, 0, 4, 0, 3, 4, 0, 2, 0, 2, 1, 0,
       2, 3, 0, 3, 3, 0, 2, 2, 3, 2, 3, 3, 2, 1, 1, 1, 2, 1, 1, 3, 0, 2, 0,
       4, 0, 0, 3, 4, 3, 1, 0, 2, 0, 1, 3, 0, 2, 1, 2, 2, 1, 4, 3, 2, 2, 3,
       4, 3, 0, 4, 0, 0, 1, 3, 3, 1, 2, 3, 2, 1, 3, 2, 0, 3, 2, 3, 3, 4, 2,
       1, 3,

In [39]:
len(pos)

534

In [42]:
posts[1]

' 인공지능 레벨1. 가전제품을 이용한 단순 제어 프로그램2. 장기나 청소로봇 같은 다채로운 프로그램3. 빅데이터를 이용한 프로그램4. 기계학습, 딥러닝을 이용한 프로그램제1차 AI 붐 추론, 탐색의 시대제2차 AI 붐 지식의 시대제3차 AI 붐 기계학습, 특징 표현 학습미로 풀기 -> 막다른 길에 다다르면 전의 코너에서 이동히노이의 탑 -> 작은 원반 위 큰 원반을 올릴 수 없다.(원반마다 가중치나 순위를 매겨 정하면 될듯)체스 -> 최소 최대 법 (2수 앞 예측)1. 경우의 수를 수치로 표현2. 경우의 수의 수치가 최소인 것 선택3. 최소중 가장 나은 것 선택몬테 카를로법 -> 수의 계산을 포기하고 랜덤으로 둔다사야 프로젝트 -> 노드를 이용해 관계 표시추이율 성립 -> 1<3, 3<7, 1<7 (O)(is-a) -> 가위<바위, 바위<보, 보<가위 (X)헤비 웨이트 온톨로지 -> 지식을 기술 방법, 인간이 직접 개입라이트 웨이트 온톨로지 -> 지식을 기술 방법, 컴퓨터가 자동으로심볼 그라운딩 문제 -> 볼과 그것을 의미하는 것이 결부프레임 문제로봇, 동굴, 배터리, 시한폭탄로봇 1: 동굴에서 배터리를 가져옴 (시한폭탄도 같이 가져옴)로봇 2: 부차적으로 일어나는 것도 고려 (동굴에서 생각하다 터짐)로봇 3: 관계없는 사항은 고려 않도록 (동굴에도 못 들어감)기계학습(학습하다->분류하다)1. 지도학습-> 입력화 출력2. 비지도학습 -> 데이터만 입력 (패턴을 추출)1. 최근접 이웃 방법->가장 가까운 이웃의 분류에 따른다.단점: 정치 카테고리에 문화가 섞이면 오버 피팅 결과2. 나이브 베이즈 법->베이즈의 정리 사용(단어가 포함되는 확률로 카테고리별 점수를 매겨 분류)3. 결정 트리->판단 기준으로 O, X를 이용해 판별단점: 복잡한 문제에서 정밀도가 높지 않다.4. 서포트 벡터 머신->데이터를 구분 짓는 구분선과 그룹 간의 간격을 최대로 나눈다.장/단점: 정밀도가 높다. 큰 데이터는 오래 걸린다.5. 뉴럴 네트워크->인간의 뇌신경을 흉내 임계값 기준으로 

In [43]:
posts[3]

' Deep learning 을 위한 TensorFlow 텐서플로우는 원래 구글의 인공지능 관련 연구팀인 Google brain팀에서 소속 연구원 및 엔지니어들이 머신러닝, 딥러닝 연구를 목적으로 만들어진 라이브러리이다. 텐서플로우는 노드(Node)와 엣지(Edge)로 구성된 Data flow graph 형태의 구조로 수치 계산을 위해 사용된다. 그래프의 노드는 수학적 연산 역할을 하고, 노드 사이에 연결된 엣지는 연결된 노드들 사이에서 전달되는 텐서(다차원 데이터 배열)의 역할을 한다. 텐서플로우는 별도의 라이브러리 없이 하나 이상의 CPU와 GPU를 적절히 분산시켜 사용할 수 있다는 것이 큰 장점이다.(https://www.tensorflow.org) 텐서플로우를 이용하는 세계적인 기업들 텐서플로우의 아키텍쳐를 살펴보자 (출처:https://www.tensorflow.org/extend/architecture)텐서플로우 런타임은 cross-platform 라이브러리로 C++, Python, Java, GO 등 여러 언어에서 사용할 수 있다.구조는 역할에 따라 크게 Client, Distributed master, Worker Service, Kernel implementation 으로 나눠진다.Client는 수행해야할 계산을 그래프의 흐름으로써 정의한다. 그리고 세션을 이용해 그래프 계산을 시작한다. Distributed master는 부분 그래프 계산 작업을 서로 다른 장치 및 프로세서에서 실행될 수 있도록 분할 해준다.즉, 그래프의 부분을 다시 여러 태스크로 나눠 일을 분배해주는 것이다. 일을 쪼개고 나면 각 Worker service에게 일을 나눠서 배포한다.그리고 Worker service의 일을 시작하게 명령하는 역할도 담당한다.Worker service는 일이 할당되면 사용가능한 하드웨어(CPU,GPU 등)에 적절한 Kernel implementation을 사용하여 할당 받은 그래프 작업을 실행할 수 있도록 스케줄링한다.Kernel impleme